In [1]:
!pip install ultralytics
!pip install numpy
!pip install playsound

^C


  Using cached numpy-1.22.4-cp39-cp39-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.4 which is incompatible.


  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7035 sha256=6f2cc13f30f8ae523ddd174cf20cc186946f830409cd6ec4fa53058a9db92fc1
  Stored in directory: c:\users\artem\appdata\local\pip\cache\wheels\ba\39\54\c8f7ff9a88a644d3c58b4dec802d90b79a2e0fb2a6b884bf82
Successfully built playsound


In [1]:
#import tensorflow as tf
#import torch
from ultralytics import YOLO

#model = torch.load('/content/drive/MyDrive/mewingAI/best.pt')
model = YOLO('best.pt')
#model.load('/content/drive/MyDrive/mewingAI/best.pt')

Prediction
https://docs.ultralytics.com/modes/predict/#inference-arguments

In [19]:
#@title some helper functions

def predict(path):
  results = model(path)
  result = results[0]
  boxes = result.boxes  # Boxes object for bounding box outputs
  masks = result.masks  # Masks object for segmentation masks outputs
  keypoints = result.keypoints  # Keypoints object for pose outputs
  probs = result.probs  # Probs object for classification outputs
  obb = result.obb  # Oriented boxes object for OBB outputs
  #result.show()  # display to screen


  #result.save(filename="result.jpg")  # save to disk
  return result.tojson(normalize=True)
  #return probs
#thing = predict("/content/drive/MyDrive/mewingAI/datasets/FacialFeatures_split/test/images/00a0fd8177a1db74_jpg.rf.e66cca8cc8cccf937b160ea6546b1fbb.jpg")

import numpy as np
import cv2

def preprocess_input(image_pil, net_h=416, net_w=416):
    image = np.asarray(image_pil)
    new_h, new_w, _ = image.shape
    # print("net:", net_h, net_w)
    # print("old:",new_h, new_w)
    # determine the new size of the image
    if (float(net_w)/new_w) < (float(net_h)/new_h):
        new_h = (new_h * net_w)/new_w
        new_w = net_w
    else:
        new_w = (new_w * net_h)/new_h
        new_h = net_h
    new_w = int(new_w)
    new_h = int(new_h)
    # print("new:",int(new_h), int(new_w))
    # resize the image to the new size
    #resized = cv2.resize(image[:,:,::-1]/255., (int(new_w), int(new_h)))
    resized = cv2.resize(image/255., (int(new_w), int(new_h)))

    # embed the image into the standard letter box
    # print("dims:",int((net_h-new_h)//2), int((net_h+new_h)//2), int((net_w-new_w)//2), int((net_w+new_w)//2))
    new_image = np.ones((net_h, net_w, 3)) * 0.5
    new_image[int((net_h-new_h)//2):int((net_h+new_h)//2), int((net_w-new_w)//2):int((net_w+new_w)//2), :] = resized
    new_image = np.expand_dims(new_image, 0)
    # print(new_image.shape)

    return new_image

In [13]:
def getResults(img):
  test = processImg(img)
  someJson = predict(test)
  result = readOutput(someJson)
  return result

#getResults(take_photo())
import json
from PIL import Image
from matplotlib import  pyplot as plt
def processImg(path):
  image_path = path
  image_pil = Image.open(image_path)
  image_w, image_h = image_pil.size
  new_image = preprocess_input(image_pil, 640, 640)

  new_image_pil = Image.fromarray((new_image[0] * 255).astype('uint8'))
  #new_image_pil.save("/content/new_img.jpg")

  #plt.imshow(new_image[0])
  #plt.show()

  return new_image_pil

def readOutput(vals):
  jsonVals=json.loads(vals)
  names = []
  confidence=[]
  for item in jsonVals:
    #print(item)
    names.append(item['name'])
    confidence.append(item['confidence'])
  print(names)
  print(confidence)

  if ('openmouth' in names) and ('closedmouth' in names):
    openIndex=names.index('openmouth')
    closedIndex=names.index('closedmouth')
    if confidence[openIndex] > confidence[closedIndex]:
      return 'open'
    else:
      return 'closed'
  elif 'openmouth' in names:
    return 'open'
  else:
    return 'closed'

In [22]:

getResults("WIN_20240722_20_03_10_Pro.jpg")


0: 640x640 1 closedmouth, 1 face, 2 openeyes, 204.7ms
Speed: 3.0ms preprocess, 204.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye', 'closedmouth']
[0.87849, 0.65566, 0.60965, 0.54173]


'closed'

In [42]:
!pip install opencv-python
!pip install playsound

In [20]:
import cv2

cam = cv2.VideoCapture(0)
cv2.namedWindow("test")



In [43]:
#while True:
from cv2 import *
from playsound import playsound

while True:
    result,image = cam.read()
    cv2.imwrite("image.png", image) 
    results = getResults("image.png")

    if results == 'open':
      print("open")



0: 640x640 1 closedmouth, 1 face, 2 openeyes, 196.2ms
Speed: 4.0ms preprocess, 196.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye', 'closedmouth']
[0.85173, 0.54424, 0.5078, 0.31209]

0: 640x640 1 closedmouth, 1 face, 2 openeyes, 194.2ms
Speed: 3.0ms preprocess, 194.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye', 'closedmouth']
[0.83722, 0.53474, 0.52041, 0.27643]

0: 640x640 1 face, 2 openeyes, 187.2ms
Speed: 3.0ms preprocess, 187.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye']
[0.84275, 0.54802, 0.51253]

0: 640x640 1 closedmouth, 1 face, 2 openeyes, 188.2ms
Speed: 2.0ms preprocess, 188.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye', 'closedmouth']
[0.83358, 0.55116, 0.53451, 0.35886]

0: 640x640 1 closedmouth, 1 face, 2 openeyes, 188.2ms
Speed: 2.0ms preprocess, 188.2ms inferenc

KeyboardInterrupt: 

Display image

In [ ]:
#@title also starts mewing

import time
#picture = take_photo_auto()

# Process the picture
#results = getResults(picture)

while True:
  picture = take_photo_auto()

  # Process the picture
  results = getResults(picture)

  # Print or log the results
  if results == "open":
    #playsound('noise.mp3')
    output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
    #print("hi")

  # Wait for the next interval

<IPython.core.display.Javascript object>


0: 640x640 2 closedmouths, 1 face, 2 openeyes, 13.6ms
Speed: 2.0ms preprocess, 13.6ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye', 'closedmouth', 'closedmouth']
[0.87041, 0.57583, 0.55297, 0.49102, 0.30749]


<IPython.core.display.Javascript object>


0: 640x640 2 closedmouths, 1 face, 2 openeyes, 7.4ms
Speed: 1.5ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye', 'closedmouth', 'closedmouth']
[0.87928, 0.61969, 0.61176, 0.5079, 0.27269]


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>


0: 640x640 2 closedmouths, 1 face, 3 openeyes, 9.3ms
Speed: 2.0ms preprocess, 9.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye', 'closedmouth', 'closedmouth', 'openeye']
[0.86757, 0.55983, 0.55042, 0.5123, 0.27753, 0.27611]


<IPython.core.display.Javascript object>


0: 640x640 2 closedmouths, 1 face, 2 openeyes, 8.8ms
Speed: 1.8ms preprocess, 8.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'closedmouth', 'openeye', 'closedmouth']
[0.87094, 0.61497, 0.59628, 0.54837, 0.35455]


<IPython.core.display.Javascript object>